In [1]:
import cv2 as cv
import numpy as np
import time
import os
import mediapipe as mp
import math
import autopy

In [6]:
# Hand Traching Module
class handDetector():
    def __init__(self , mode = False , maxHands = 2 , detectionCon = 0.5 , trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpHands= mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode , self.maxHands ,1, self.detectionCon , self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
        self.tipIds = [4 , 8 , 12 , 16 , 20]
        
    def findHands(self,img,draw = True):
        imgRGB =  cv.cvtColor(img , cv.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw : self.mpDraw.draw_landmarks(img,handLms , self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self , img , handNo = 0 , draw = True):
        self.lmList = []
        if self.results.multi_hand_landmarks:
            
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id , lm in enumerate(myHand.landmark):
                    h, w ,c = img.shape
                    cx , cy = int(lm.x*w ) , int(lm.y * h)
                    self.lmList.append([id, cx, cy])
                    
                    if draw : cv.circle(img,(cx , cy) , 5 , (255,0,255) , cv.FILLED)
                    
        return self.lmList
    
    def fingersUp(self):
        fingers = []
        
        # thums
        if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(0)
        else:
            fingers.append(1)
            
        # Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
                
        return fingers
    
    def findDistance(self , p1 , p2 , img , draw = True , r = 15 , t = 3):
        x1 , y1 = self.lmList[p1][1:]
        x2 , y2 = self.lmList[p2][1:]
        cx , cy = (x1 + x2) // 2 , (y1 + y2) // 2 
        
        if draw:
            cv.line(img,(x1,y1),(x2,y2), (255,0,255) , t)
            cv.circle(img,(x1,y1),r , (255,0,255) , cv.FILLED)
            cv.circle(img,(x2,y2),r , (255,0,255) , cv.FILLED)
            cv.circle(img,(cx,cy),r , (0,0,255) , cv.FILLED)
        length = math.hypot(x2 - x1 , y2 - y1)
        
        
        return length , img , [x1 , x2 , y1 , y2 , cx , cy]

In [16]:
wCam ,hCam = 640 , 480
wScr , hScr = autopy.screen.size()

cap = cv.VideoCapture(0)
cap.set(3 , wCam)
cap.set(4 , hCam)

detector = handDetector(maxHands=1)
pTime = 0
frameR = 100  # Frame reduction
smoothening = 20
pLocX , pLocY = 0,0
cLocX , cLocY = 0,0

while cap.isOpened():
    _ , frame = cap.read()
    frame = detector.findHands(frame)
    lmList = detector.findPosition(frame)
    
    # Get the tip of the index and middle fingers
    if len(lmList) != 0:
        x1 , y1 = lmList[8][1:]
        x2 , y2 = lmList[12][1:]
        
        # Check up fingers
        fingers = detector.fingersUp()
        
        # Only index finger : Moving mode
        if fingers[1] == 1 and fingers[2] == 0:
            print("Moving Mode")
            cv.rectangle(frame , (frameR , frameR) ,(wCam-frameR , hCam-frameR), (0,255,0) , 2)
                
            # Convert Coordinates
            x3 = np.interp(x1 , (frameR,wCam-frameR) , (0 , wScr))
            y3 = np.interp(y1 , (frameR,hCam-frameR) , (0 , hScr))
            
            # Smoothen values
            cLocX = pLocX + (x3 - pLocX) / smoothening
            cLocY = pLocY + (y3 - pLocY) / smoothening
            
            
            # Move mouse
            autopy.mouse.move(wScr - cLocX , cLocY)
            cv.circle(frame , (x1 , y1) , 15 , (255,0,255) , cv.FILLED)
            pLocX , pLocY = cLocX , cLocY
            
        # Both fingers are up : Clicking mode
        elif fingers[1] == 1 and fingers[2] == 1:
            print('Clicking Mode')
            
            length , frame , lineInfo =detector.findDistance(8 , 12 , frame)

            # Find distance between fingers
            if length < 40:
                cv.circle(frame , lineInfo[4:] , 15 , (0,255,0) , cv.FILLED)
                
                # Click mouse if distance short
                autopy.mouse.click()
        
    # Frame rate
    cTime = time.time()
    fbs = 1 / (cTime - pTime)
    pTime = cTime
    cv.putText(frame,f"FBS : {int(fbs)}" , (20,70),cv.FONT_HERSHEY_PLAIN , 3 , (0,255,0))
    
    cv.imshow("Frame" , frame)
    if cv.waitKey(1) == ord('q'):
       break
   
    
cap.release()
cv.destroyAllWindows()

d:\Src\Anaconda\lib\site-packages\google\protobuf\symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Clicking Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving Mode
Moving